# Import libraries

In [ ]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
import itertools
from sklearn.model_selection import train_test_split
from scipy import stats
from tqdm import tqdm

In [ ]:
import pandas as pd, numpy as np
from statistics import mode, StatisticsError
from collections import defaultdict
from operator import itemgetter

#Import data 

In [ ]:
df_train = pd.read_csv("/content/Cleaned_train_data.csv", index_col="seq_id")



In [ ]:
# drop nan values
df_train = df_train.dropna(subset=["pH"])

In [ ]:
df_train

protein_sequence   pH  \
seq_id                                                           
0       AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVG...  7.0   
1       AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSR...  7.0   
2       AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYA...  7.0   
3       AAASGLRTAIPAQPLRHLLQPAPRPCLRPFGLLSVRAGSARRSGLL...  7.0   
5       AACFWRRTVIPKPPFRGISTTSARSTVMPAWVIDKYGKNEVLRFTQ...  7.0   
...                                                   ...  ...   
31385   YYMYSGGGSALAAGGGGAGRKGDWNDIDSIKKKDLHHSRGDEKAQG...  7.0   
31386   YYNDQHRLSSYSVETAMFLSWERAIVKPGAMFKKAVIGFNCNVDLI...  7.0   
31387   YYQRTLGAELLYKISFGEMPKSAQDSAENCPSGMQFPDTAIAHANV...  7.0   
31388   YYSFSDNITTVFLSRQAIDDDHSLSLGTISDVVESENGVVAADDAR...  7.0   
31389   YYVPDEYWQSLEVAHKLTFGYGYLTWEWVQGIRSYVYPLLIAGLYK...  7.0   

                              data_source    tm  length  
seq_id                                                   
0       doi.org/10.1038/s41592-020-0801-4  75.7     341  
1       doi.org/10.1038/s41592-020-0801-4  50.5     286  
2       doi.org/10.1038/s41592-020-0801-4  40.5     497  
3       doi.org/10.1038/s41592-020-0801-4  47.2     265  
5       doi.org/10.1038/s41592-020-0801-4  48.4     380  
...                                   ...   ...     ...  
31385   doi.org/10.1038/s41592-020-0801-4  51.8     549  
31386   doi.org/10.1038/s41592-020-0801-4  37.2     469  
31387   doi.org/10.1038/s41592-020-0801-4  64.6     128  
31388   doi.org/10.1038/s41592-020-0801-4  50.7     593  
31389   doi.org/10.1038/s41592-020-0801-4  37.6     537  

[26855 rows x 5 columns]

#Data Exploration

In [ ]:
df_train['length'] = df_train.protein_sequence.str.len()
df_train['length'].head()
#Counting the length of different proteins

seq_id
0    341
1    286
2    497
3    265
5    380
Name: length, dtype: int64

In [ ]:
df_train['length'].describe()

count    26855.000000
mean       372.578589
std        214.495301
min          5.000000
25%        200.000000
50%        333.000000
75%        487.000000
max       1024.000000
Name: length, dtype: float64

In [ ]:
Unique_values = df_train['length'].value_counts() #Find the unique values for length and count these
Unique_values.head(10)


164    748
231    318
455    245
155    243
148    241
448    180
170    165
150    164
96     145
142    125
Name: length, dtype: int64

In [ ]:
Unique_values.describe()


count    1002.000000
mean       26.801397
std        34.686976
min         1.000000
25%         9.000000
50%        19.000000
75%        39.000000
max       748.000000
Name: length, dtype: float64

In [ ]:
Unique_values.gt(6).value_counts() 

True     828
False    174
Name: length, dtype: int64

In [ ]:
Unique_values.gt(5).value_counts() #872 lenghts have count greater than 5

True     872
False    130
Name: length, dtype: int64

In [ ]:
Unique_values.gt(4).value_counts()

True     912
False     90
Name: length, dtype: int64

In [ ]:
Unique_values.gt(3).value_counts()

True     940
False     62
Name: length, dtype: int64

In [ ]:
Unique_values.gt(2).value_counts()

True     971
False     31
Name: length, dtype: int64

In [ ]:
Unique_values.gt(1).value_counts()

True     984
False     18
Name: length, dtype: int64

In [ ]:
Unique_values.gt(0).value_counts()

True    1002
Name: length, dtype: int64

##Find the wild groups

https://www.kaggle.com/code/roberthatch/novo-train-data-contains-wildtype-groups/notebook

In [59]:
Baseline = 1 #baseline of insertion, deletion is 1
Smallest = 5 #smallest group size is 5 

def count(seq):
    defaultvalue = defaultdict(int) #start a default dictionary from collections library
    for item in seq: #go through whole sequence, find the similar items
        defaultvalue[item] += 1
    return max(defaultvalue.items(), key=itemgetter(1)) #return the value and count of most frequent item

In [60]:
def Findwildtype(proteinsequence, is_retry=False): #Defined function to find wildtype
    if not is_retry:
        wildtype = []
        try:  #Go throigh whole protein sequence
            for i in range(len(proteinsequence.iloc[0])):
                #Find the mode of protein in whole sequence to get the most common protein
                wildtype.append(mode([protein[i] for protein in proteinsequence]))
            return ''.join(wildtype)
        except StatisticsError:
            pass
    #Two sequence from single wild type has baseline mutation of 1 only 
    #hence it is expected that there 1/3 of consecutive length should be similar
    size = len(proteinsequence.iloc[0])//3 #divide the whole protein sequence into three part
    starts = []
    middles = []
    ends = []
    for protein in proteinsequence: 
      #go through whole sequence list, divide into start, middle, center
      starts.append(protein[:size]) #make a list of start portion of all proteinsequence, 
      #do same for center and end
      middles.append(protein[size:2*size])
      ends.append(protein[-size:])
    
    
    
    start = count(starts) #apply count function mentioned above to find the most common sequence outof all
    #also find its count 
    middle = count(middles)
    end = count(ends)
   
    
   


    #now compared if for a particular proteinsequence if start[1] = count > then middle[1] count, and end one count
    #and greater than smallest group size
    #Reduce the protein to just the one which match the most common string
    if (start[1] >= middle[1]) and (start[1] >= end[1]) and (start[1] >= Smallest):
        proteinsequence = [protein for protein in proteinsequence if protein[:size] == start[0]]
        assert(start[1] == len( proteinsequence))
    elif (middle[1] >= end[1]) and (middle[1] >= Smallest):
        proteinsequence = [protein for protein in proteinsequence if protein[size:2*size] == middle[0]]
        assert(middle[1] == len( proteinsequence))
    elif end[1] >= Smallest:
        proteinsequence = [protein for protein in proteinsequence if protein[-size:] == end[0]]
        assert(end[1] == len( proteinsequence))
    else:
        return ''
    #when got the reduced list for all proteinsequences than find the most common protein through
    #that is the wildtype
    wildtype = []
    try:
        for i in range(len(proteinsequence[0])):
            wildtype.append(mode([protein[i] for protein in proteinsequence]))
        return ''.join(wildtype)
    except StatisticsError:
        return ""


In [63]:
df_train['group'] = -1 #start with a default coloumn to save integers
df_train['wildtype'] = '' #start with a default coloumn to save string of wildtypes
grp = 0

Smallest = 5
for k in range(len(Unique_values)): #go through whole uniquevalues series
    if Unique_values.iloc[k] < Smallest:  #check the condition for being greater than equal to five
        break
    c = Unique_values.index[k] #Find out the index for count that is length
    print(f'rows={Unique_values.iloc[k]}, k:{k}, protein length:{c}')
    is_retry = False
    #Find the subset with all protein same length and which does not have a group yet
    tmp = df_train.loc[(df_train.length ==c)&(df_train.group==-1)]

    
    while len(tmp) >= Smallest:
        if len(tmp)<=1: break
        
        proteins = tmp.protein_sequence.drop_duplicates()
        
        #check for all the seqeuces of same length and create wildtype
        #Use the subset to find wildtype as defined by function above
        #If could not find wildtype break and then go above again
        wildtype = Findwildtype(proteins, is_retry=is_retry)
        if wildtype == '':
            break

        #subset of data with same length but with the margin of +1 or -1 
        #to deal with insertion or deletion
        tmp = df_train.loc[(df_train.length>=c-1)&(df_train.length<=c+1)&(df_train.group==-1)]
        for idx in tmp.index:
            p = df_train.loc[idx, 'protein_sequence']
            half = c//2
            #if half of wildtyp is similar to half of p in the beginning or end, add this to same group
            #Its fine to that because in that case it will be only 1 point away, from basic proteinsequenc
            #as single mutation
            if (wildtype[:half] == p[:half]) or (wildtype[-half:] == p[-half:]):
                df_train.loc[idx,'group'] = grp
                df_train.loc[idx,'wildtype'] = wildtype
        if len(df_train.loc[df_train.group==grp]) >= Smallest:
            print(f"{df_train.loc[(df_train.group==grp)].shape[0]}: Group {grp} results")
            grp += 1
            is_retry = False
        else:
            df_train.loc[idx,'group'] = -1
            df_train.loc[idx,'wildtype'] = ''
            ## to avoid an infinite loop, break out if we've already failed last time
            if is_retry:
                break
            is_retry = True

        # Get ready for next loop
        tmp = df_train.loc[(df_train.length==c)&(df_train.group==-1)]

rows=748, k:0, protein length:164
708: Group 0 results
rows=318, k:1, protein length:231
273: Group 1 results
rows=245, k:2, protein length:455
211: Group 2 results
rows=243, k:3, protein length:155
180: Group 3 results
34: Group 4 results
rows=241, k:4, protein length:148
194: Group 5 results
rows=180, k:5, protein length:448
77: Group 6 results
61: Group 7 results
rows=165, k:6, protein length:170
144: Group 8 results
rows=164, k:7, protein length:150
124: Group 9 results
rows=145, k:8, protein length:96
127: Group 10 results
rows=125, k:9, protein length:142
78: Group 11 results
rows=123, k:10, protein length:101
78: Group 12 results
32: Group 13 results
rows=116, k:11, protein length:109
78: Group 14 results
rows=115, k:12, protein length:485
84: Group 15 results
rows=99, k:13, protein length:144
68: Group 16 results
rows=99, k:14, protein length:268
53: Group 17 results
rows=99, k:15, protein length:537
72: Group 18 results
rows=94, k:16, protein length:163
60: Group 19 results
ro

In [62]:
print(grp)

75


https://stackoverflow.com/questions/3071415/efficient-method-to-calculate-the-rank-vector-of-a-list-in-python, method to calculate rank of vetor

In [ ]:
def argsort(seq, reverse=False):
    # http://stackoverflow.com/questions/3071415/efficient-method-to-calculate-the-rank-vector-of-a-list-in-python
    return sorted(range(len(seq)), key=seq.__getitem__, reverse=reverse)

groups = [0] * grp 
for k in range(grp):
    groups[k] = len(df_train.loc[df_train.group==k])

groupCount = 0
rowCount = 0
for k in argsort(groups, reverse=True):
    if df_train.loc[df_train.group==k].shape[0] == 0:
        continue
    proteins = df_train.loc[df_train.group==k, "protein_sequence"]
    wildtype = df_train.loc[df_train.group==k, "wildtype"].values[0]

   
    for i in range(len(proteins)):
        if len(proteins.iloc[i]) < len(proteins.iloc[0]):
            if proteins.iloc[i] == wildtype[:-1]:
                proteins.iloc[i] = proteins.iloc[i] + "-"
            else:
                for j in range(len(proteins.iloc[i])):
                    if proteins.iloc[i][j] != wildtype[j]:
                        proteins.iloc[i] = proteins.iloc[i][:j-1] + "-" + proteins.iloc[i][j:]
                        break
        assert(len(proteins.iloc[i]) == len(proteins.iloc[0]))

   
    ungroup = []
    for p in proteins:
        mut = 0
        for j in range(len(wildtype)):
            if p[j] != wildtype[j]:
                mut += 1
        if mut > 1:
            if p not in ungroup:
                ungroup.append(p)
    for p in ungroup:
        df_train.loc[df_train.protein_sequence==p, 'group'] = -1
        df_train.loc[df_train.protein_sequence==p, 'wildtype'] = ''
   
    if df_train.loc[df_train.group==k].shape[0] < Smallest:
       df_train.loc[df_train.group==k, 'wildtype'] = ''
       df_train.loc[df_train.group==k, 'group'] = -1
       continue

    ## Print a line for every group, and a bunch of stats for the first few groups
    print(f'{k}: {df_train.loc[df_train.group==k].shape[0]}')
    if groupCount < 5:
        display( df_train.loc[df_train.group==k] )
        for c in df_train.columns:
            print(c, df_train.loc[df_train.group==k, c].nunique() + df_train.loc[df_train.group==k, c].isnull().values.any())
        print(wildtype)
        print("")
    groupCount += 1
    rowCount += df_train.loc[df_train.group==k].shape[0]

print(groupCount, rowCount)

0: 708


protein_sequence   pH  \
seq_id                                                           
18020   MNAFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...  2.0   
18021   MNAFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...  4.2   
18022   MNAFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...  2.0   
18023   MNAFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...  6.5   
18060   MNCFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...  2.0   
...                                                   ...  ...   
19949   MNWFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...  6.5   
19950   MNWFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...  2.0   
19964   MNYFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...  2.0   
19965   MNYFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...  6.5   
19966   MNYFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...  4.2   

                data_source    tm  length  group  \
seq_id                                             
18020   10.1021/bi00535a054  38.1     164      0   
18021                   NaN  53.3     164      0   
18022      10.1038/334406a0  38.1     164      0   
18023      10.1038/334406a0  62.9     164      0   
18060                   NaN  41.9     164      0   
...                     ...   ...     ...    ...   
19949      10.1038/334406a0  56.7     164      0   
19950      10.1038/334406a0  25.5     164      0   
19964      10.1038/334406a0  32.4     164      0   
19965      10.1038/334406a0  58.8     164      0   
19966                   NaN  53.3     164      0   

                                                 wildtype  
seq_id                                                     
18020   MNIFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...  
18021   MNIFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...  
18022   MNIFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...  
18023   MNIFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...  
18060   MNIFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...  
...                                                   ...  
19949   MNIFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...  
19950   MNIFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...  
19964   MNIFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...  
19965   MNIFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...  
19966   MNIFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...  

[708 rows x 7 columns]

protein_sequence 292
pH 54
data_source 59
tm 295
length 1
group 1
wildtype 1
MNIFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSELDKAIGRNCNGVITKDEAEKLFNQDVDAAVRGILRNAKLKPVYDSLDAVRRCALINMVFQMGETGVAGFTNSLRMLQQKRWDEAAVNLAKSIWYNQTPNRAKRVITTFRTGTWDAYKNL

1: 273


protein_sequence    pH  \
seq_id                                                            
16542   MLVMTEYLLSAGICMAIVSILLIGMAISNVSKGQYAKRFFFFATSC...  7.00   
16544   MLVMTEYLLSAGICMAIVSILLIGMAISNVSKGQYAKRFFFFATSC...  5.40   
16554   MLVMTEYLLSAGICMAIVSILLIGMAISNVSKGQYAKRFFFFATSC...  7.00   
16555   MLVMTEYLLSAGICMAIVSILLIGMAISNVSKGQYAKRFFFFATSC...  7.00   
16559   MLVMTEYLLSAGICMAIVSILLIGMAISNVSKGQYAKRFFFFATSC...  7.00   
...                                                   ...   ...   
17299   MLVMTEYLLSAGICMAIVSILLIGMAISNVSKGQYAKRFFFFATSC...  7.00   
17304   MLVMTEYLLSAGICMAIVSILLIGMAISNVSKGQYAKRFFFFATSC...  5.42   
17306   MLVMTEYLLSAGICMAIVSILLIGMAISNVSKGQYAKRFFFFATSC...  5.40   
17314   MLVMTEYLLSAGICMAIVSILLIGMAISNVSKGQYAKRFFFFATSC...  7.00   
17316   MLVMTEYLLSAGICMAIVSILLIGMAISNVSKGQYAKRFFFFATSC...  7.00   

                      data_source    tm  length  group  \
seq_id                                                   
16542         10.1021/bi00006a025  50.9     231      1   
16544                         NaN  40.0     231      1   
16554   10.1016/j.bpc.2006.10.014  50.1     231      1   
16555   10.1016/j.bpc.2006.10.014  48.9     231      1   
16559   10.1016/j.bpc.2006.10.014  51.1     231      1   
...                           ...   ...     ...    ...   
17299   10.1016/j.bpc.2006.10.014  48.5     231      1   
17304         10.1021/bi00471a003  45.7     231      1   
17306                         NaN  40.0     231      1   
17314   10.1016/j.bpc.2006.10.014  52.4     231      1   
17316   10.1016/j.bpc.2006.10.014  51.1     231      1   

                                                 wildtype  
seq_id                                                     
16542   MLVMTEYLLSAGICMAIVSILLIGMAISNVSKGQYAKRFFFFATSC...  
16544   MLVMTEYLLSAGICMAIVSILLIGMAISNVSKGQYAKRFFFFATSC...  
16554   MLVMTEYLLSAGICMAIVSILLIGMAISNVSKGQYAKRFFFFATSC...  
16555   MLVMTEYLLSAGICMAIVSILLIGMAISNVSKGQYAKRFFFFATSC...  
16559   MLVMTEYLLSAGICMAIVSILLIGMAISNVSKGQYAKRFFFFATSC...  
...                                                   ...  
17299   MLVMTEYLLSAGICMAIVSILLIGMAISNVSKGQYAKRFFFFATSC...  
17304   MLVMTEYLLSAGICMAIVSILLIGMAISNVSKGQYAKRFFFFATSC...  
17306   MLVMTEYLLSAGICMAIVSILLIGMAISNVSKGQYAKRFFFFATSC...  
17314   MLVMTEYLLSAGICMAIVSILLIGMAISNVSKGQYAKRFFFFATSC...  
17316   MLVMTEYLLSAGICMAIVSILLIGMAISNVSKGQYAKRFFFFATSC...  

[273 rows x 7 columns]

protein_sequence 143
pH 17
data_source 17
tm 153
length 1
group 1
wildtype 1
MLVMTEYLLSAGICMAIVSILLIGMAISNVSKGQYAKRFFFFATSCLVLTLVVVSSLSSSANASQTDNGVNRSGSEDPTVYSATSTKKLHKEPATLIKAIDGDTVKLMYKGQPMTFRLLLVDTPETKHPKKGVEKYGPEASAFTKKMVENAKKIEVEFDKGQRTDKYGRGLAYIYADGKMVNEALVRQGLAKVAYVYKPNNTHEQHLRKSEAQAKKEKLNIWSEDNADSGQ

2: 211


...

protein_sequence 211
pH 1
data_source 1
tm 31
length 1
group 1
wildtype 1
MNQSVSSLPEKDIQYQLHPYTNARLHQELGPLIIERGEGIYVYDDQGKGYIEAMAGLWSAALGFSNQRLIKAAEQQFNTLPFYHLFSHKSHRPSIELAEKLIEMAPVPMSKVFFTNSGSEANDTVVKMVWYLNNALGKPAKKKFISRVNGYHGITVASASLTGLPGNQRGFDLPLPGFLHVGCPHHYRFALAGESEEHFADRLAVELEQKILAEGPETIAAFIGEPLMGAGGVIVPPRTYWEKIQKVCRKYDILVIADEVICGFGRTGQMFGSQTFGIQPDIMVLSKQLSSSYQPIAAILINAPVFEGIADQSQALGALGHGFTGSGHPVATAVALENLKIIEEESLVEHAAQMGQLLRSGLQHFIDHPLVGEIRGCGLIAAVELVGDRVSKAPYQALGTLGRYMAGRAQEHGMITRAMGDAVAFCPPLIVNEQEVGMIVERFARALDDTTQWVG

5: 194


protein_sequence   pH  \
seq_id                                                           
13269   MKALIVLGLVLLSVTVQGAVFERCELARTLKRLGMDGYRGISLANW...  2.7   
13270   MKALIVLGLVLLSVTVQGKAFERCELARTLKRLGMDGYRGISLANW...  2.7   
13271   MKALIVLGLVLLSVTVQGKAFERCELARTLKRLGMDGYRGISLANW...  2.7   
13273   MKALIVLGLVLLSVTVQGKDFERCELARTLKRLGMDGYRGISLANW...  2.7   
13275   MKALIVLGLVLLSVTVQGKFFERCELARTLKRLGMDGYRGISLANW...  2.7   
...                                                   ...  ...   
13569   MKALIVLGLVLLSVTVQGKVFERCQLARTLKRLGMDGYRGISLANW...  2.2   
13570   MKALIVLGLVLLSVTVQGKVFERCQLARTLKRLGMDGYRGISLANW...  4.0   
13571   MKALIVLGLVLLSVTVQGKVFERCQLARTLKRLGMDGYRGISLANW...  2.3   
13573   MKALIVLGLVLLSVTVQGKYFERCELARTLKRLGMDGYRGISLANW...  2.7   
13574   MKALIVLGLVLLSVTVQGMVFERCELARTLKRLGMDGYRGISLANW...  2.7   

                             data_source    tm  length  group  \
seq_id                                                          
13269   10.1046/j.1432-1327.1999.00918.x  62.7     148      5   
13270                  10.1021/bi9621829  60.3     148      5   
13271             10.1006/jmbi.1998.1906  41.3     148      5   
13273             10.1074/jbc.M110728200  60.4     148      5   
13275                  10.1021/bi0015717  62.2     148      5   
...                                  ...   ...     ...    ...   
13569                  10.1021/bi000849s  57.8     148      5   
13570                  10.1021/bi000849s  76.2     148      5   
13571                  10.1021/bi000849s  54.6     148      5   
13573             10.1074/jbc.M110728200  63.7     148      5   
13574   10.1046/j.1432-1327.1999.00918.x  64.4     148      5   

                                                 wildtype  
seq_id                                                     
13269   MKALIVLGLVLLSVTVQGKVFERCELARTLKRLGMDGYRGISLANW...  
13270   MKALIVLGLVLLSVTVQGKVFERCELARTLKRLGMDGYRGISLANW...  
13271   MKALIVLGLVLLSVTVQGKVFERCELARTLKRLGMDGYRGISLANW...  
13273   MKALIVLGLVLLSVTVQGKVFERCELARTLKRLGMDGYRGISLANW...  
13275   MKALIVLGLVLLSVTVQGKVFERCELARTLKRLGMDGYRGISLANW...  
...                                                   ...  
13569   MKALIVLGLVLLSVTVQGKVFERCELARTLKRLGMDGYRGISLANW...  
13570   MKALIVLGLVLLSVTVQGKVFERCELARTLKRLGMDGYRGISLANW...  
13571   MKALIVLGLVLLSVTVQGKVFERCELARTLKRLGMDGYRGISLANW...  
13573   MKALIVLGLVLLSVTVQGKVFERCELARTLKRLGMDGYRGISLANW...  
13574   MKALIVLGLVLLSVTVQGKVFERCELARTLKRLGMDGYRGISLANW...  

[194 rows x 7 columns]

protein_sequence 122
pH 12
data_source 20
tm 110
length 1
group 1
wildtype 1
MKALIVLGLVLLSVTVQGKVFERCELARTLKRLGMDGYRGISLANWMCLAKWESGYNTRATNYNAGDRSTDYGIFQINSRYWCNDGKTPGAVNACHLSCSALLQDNIADAVACAKRVVRDPQGIRAWVAWRNRCQNRDVRQYVQGCGV

3: 180


protein_sequence   pH  \
seq_id                                                           
15685   MLKQVEIFTAGSCLGNPGPGGYGAILRYRGREKTFSAGYTRTTNNR...  3.0   
15686   MLKQVEIFTAGSCLGNPGPGGYGAILRYRGREKTFSAGYTRTTNNR...  9.0   
15687   MLKQVEIFTAGSCLGNPGPGGYGAILRYRGREKTFSAGYTRTTNNR...  5.5   
15688   MLKQVEIFTAGSCLGNPGPGGYGAILRYRGREKTFSAGYTRTTNNR...  5.8   
15690   MLKQVEIFTDGSCLGNPGPGGYAAILRYRGREKTFSAGYTRTTNNR...  4.2   
...                                                   ...  ...   
15875   MLKQVEIFTNGSCLGNPGPGGYGAILRYRGREKTFSAGYTRTTNNR...  9.0   
15876   MLKQVEIFTNGSCLGNPGPGGYGAILRYRGREKTFSAGYTRTTNNR...  3.0   
15878   MLKQVEIFTSGSCLGNPGPGGYGAILRYRGREKTFSAGYTRTTNNR...  9.0   
15879   MLKQVEIFTSGSCLGNPGPGGYGAILRYRGREKTFSAGYTRTTNNR...  3.0   
15880   MLKQVEIFTSGSCLGNPGPGGYGAILRYRGREKTFSAGYTRTTNNR...  6.0   

                     data_source    tm  length  group  \
seq_id                                                  
15685   10.1074/jbc.271.51.32729  58.1     155      3   
15686   10.1074/jbc.271.51.32729  60.7     155      3   
15687              10.1038/12277  62.6     155      3   
15688                        NaN  40.7     155      3   
15690                        NaN  51.5     155      3   
...                          ...   ...     ...    ...   
15875   10.1074/jbc.271.51.32729  53.8     155      3   
15876   10.1074/jbc.271.51.32729  47.4     155      3   
15878   10.1074/jbc.271.51.32729  56.2     155      3   
15879   10.1074/jbc.271.51.32729  52.4     155      3   
15880                        NaN  48.5     155      3   

                                                 wildtype  
seq_id                                                     
15685   MLKQVEIFTDGSCLGNPGPGGYGAILRYRGREKTFSAGYTRTTNNR...  
15686   MLKQVEIFTDGSCLGNPGPGGYGAILRYRGREKTFSAGYTRTTNNR...  
15687   MLKQVEIFTDGSCLGNPGPGGYGAILRYRGREKTFSAGYTRTTNNR...  
15688   MLKQVEIFTDGSCLGNPGPGGYGAILRYRGREKTFSAGYTRTTNNR...  
15690   MLKQVEIFTDGSCLGNPGPGGYGAILRYRGREKTFSAGYTRTTNNR...  
...                                                   ...  
15875   MLKQVEIFTDGSCLGNPGPGGYGAILRYRGREKTFSAGYTRTTNNR...  
15876   MLKQVEIFTDGSCLGNPGPGGYGAILRYRGREKTFSAGYTRTTNNR...  
15878   MLKQVEIFTDGSCLGNPGPGGYGAILRYRGREKTFSAGYTRTTNNR...  
15879   MLKQVEIFTDGSCLGNPGPGGYGAILRYRGREKTFSAGYTRTTNNR...  
15880   MLKQVEIFTDGSCLGNPGPGGYGAILRYRGREKTFSAGYTRTTNNR...  

[180 rows x 7 columns]

protein_sequence 74
pH 11
data_source 11
tm 93
length 2
group 1
wildtype 1
MLKQVEIFTDGSCLGNPGPGGYGAILRYRGREKTFSAGYTRTTNNRMELMAAIVALEALKEHCEVILSTDSQYVRQGITQWIHNWKKRGWKTADKKPVKNVDLWQRLDAALGQHQIKWEWVKGHAGHPENERCDELARAAAMNPTLEDTGYQVEV

8: 144
10: 127
9: 124
15: 84
11: 78
12: 78
14: 78
6: 77
18: 72
16: 68
29: 62
7: 61
19: 60
28: 59
34: 55
22: 54
25: 54
17: 53
26: 53
44: 52
20: 51
21: 49
50: 44
24: 41
4: 34
13: 32
37: 31
49: 30
23: 28
33: 28
57: 27
30: 26
31: 25
42: 24
71: 21
27: 20
32: 19
38: 19
43: 19
48: 17
35: 16
40: 16
69: 13
70: 13
56: 12
60: 12
67: 12
66: 11
36: 10
41: 10
45: 10
46: 10
52: 10
54: 10
58: 10
62: 9
55: 8
59: 8
65: 8
68: 8
53: 7
39: 6
61: 6
63: 6
72: 6
73: 6
47: 5
51: 5
73 3907


In [ ]:
#Sorted group from highest to lowest
groups = [0] * grp
for k in range(grp):
    groups[k] = len(df_train.loc[df_train.group==k])

n = 10000
for k in argsort(groups, reverse=True):
    df_train.loc[df_train.group==k, "group"] = n
    n += 1

df_train.loc[df_train.group>=10000, "group"] = df_train.loc[df_train.group>=10000, "group"] - 10000
df_train.loc[df_train.group==-1, "group"] = 1000
train = df_train.sort_values(axis=0, by=['group'], kind='mergesort').reset_index(drop=True)
train.loc[train.group==1000, "group"] = -1

# train = train.drop('x',axis=1)
train_wildtype_groups = train.loc[train.wildtype != '']
train_no_wildtype = train.loc[train.wildtype == '']
print(train_wildtype_groups.shape)
print(train_no_wildtype.shape)

(3875, 7)
(22980, 7)


In [ ]:

train_wildtype_groups.head(25)


protein_sequence   pH  \
0   MNAFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...  2.0   
1   MNAFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...  4.2   
2   MNAFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...  2.0   
3   MNAFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...  6.5   
4   MNCFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...  2.0   
5   MNCFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...  6.5   
6   MNCFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...  6.5   
7   MNCFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...  4.2   
8   MNDFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...  4.2   
9   MNDFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...  6.5   
10  MNDFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...  2.0   
11  MNEFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...  2.0   
12  MNEFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...  6.5   
13  MNEFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...  3.0   
14  MNEFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...  2.5   
15  MNEFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...  2.0   
16  MNFFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...  2.0   
17  MNFFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...  6.5   
18  MNFFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...  2.5   
19  MNFFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...  3.0   
20  MNFFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...  2.0   
21  MNFFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...  2.0   
22  MNGFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...  2.0   
23  MNGFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...  2.0   
24  MNGFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...  4.2   

            data_source    tm  length  group  \
0   10.1021/bi00535a054  38.1     164      0   
1                   NaN  53.3     164      0   
2      10.1038/334406a0  38.1     164      0   
3      10.1038/334406a0  62.9     164      0   
4                   NaN  41.9     164      0   
5                   NaN  65.0     164      0   
6                   NaN  64.7     164      0   
7                   NaN  53.3     164      0   
8                   NaN  53.3     164      0   
9      10.1038/334406a0  56.2     164      0   
10     10.1038/334406a0  35.4     164      0   
11     10.1038/334406a0  37.8     164      0   
12     10.1038/334406a0  59.0     164      0   
13  10.1021/bi00159a009  49.9     164      0   
14  10.1021/bi00159a009  41.5     164      0   
15  10.1021/bi00159a009  33.1     164      0   
16  10.1021/bi00159a009  34.1     164      0   
17     10.1038/334406a0  61.7     164      0   
18  10.1021/bi00159a009  43.2     164      0   
19  10.1021/bi00159a009  52.2     164      0   
20     10.1038/334406a0  37.9     164      0   
21  10.1021/bi00535a054  37.9     164      0   
22     10.1038/334406a0  34.7     164      0   
23  10.1021/bi00535a054  34.7     164      0   
24                  NaN  53.3     164      0   

                                             wildtype  
0   MNIFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...  
1   MNIFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...  
2   MNIFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...  
3   MNIFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...  
4   MNIFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...  
5   MNIFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...  
6   MNIFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...  
7   MNIFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...  
8   MNIFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...  
9   MNIFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...  
10  MNIFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...  
11  MNIFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...  
12  MNIFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...  
13  MNIFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...  
14  MNIFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...  
15  MNIFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...  
16  MNIFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...  
17  MNIFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...  
18  MNIFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...  
19  MNIFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...  

In [ ]:
count = train_wildtype_groups.group.unique().size
print(count)

70


In [ ]:
count = train_wildtype_groups.wildtype.unique().size
print(count)

71


# Data exploration

## Check number of tokens

In [ ]:
train_sequences = df_train.iloc[:,0].to_numpy()
train_total = ''
for s in train_sequences:
    train_total = train_total + s
print(f"Number of unique characters in train protein sequence: {len(set(train_total))}")

test_sequences = df_test.iloc[:,0].to_numpy()
test_total = ''
for s in test_sequences:
    test_total = test_total + s
print(f"Number of unique characters in test protein sequence: {len(set(test_total))}")

Number of unique characters in train protein sequence: 20


NameError: ignored

## Check Tm range

In [ ]:
tms = df_train.iloc[:,3].to_numpy()
print(np.min(tms), np.max(tms))

## Check sequence length range

In [ ]:
train_lengths = np.array([len(s) for s in train_sequences])
print(np.min(train_lengths), np.max(train_lengths))

test_lengths = np.array([len(s) for s in test_sequences])
print(np.min(test_lengths), np.max(test_lengths))
print(train_lengths)
plt.hist(train_lengths, bins='auto');
plt.xlim(0, 3500)
plt.xlabel("Sequence length (training data)")
plt.ylabel("Count")
plt.show()

## Check pH range

In [ ]:
ph_train = df_train.iloc[:, 1].to_numpy()
print(np.min(ph_train), np.max(ph_train))

ph_test = df_test.iloc[:, 1].to_numpy()
print(np.min(ph_test), np.max(ph_test))

In [ ]:
# Wild type sequence provided in the "Dataset Description":
base = 'VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVDCWAILCKGAPNVLQRVNEKTKNSNRDRSGANKGPFKDPQKWGIKALPPKNPSWSAQDFKSPEEYAFASSLQGGTNAILAPVNLASQNSQGGVLNGFYSANKVAQFDPSKPQQTKGTWFQITKFTGAAGPYCKALGSNDKSVCDKNKNIAGDWGFDPAKWAYQYDEKNNKFNYVGK'

In [ ]:
listOutput = []

In [ ]:
# Base libraries



# Files management
import os

# Ploting
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

# Libraris for Data process
import blosum as bl
import Levenshtein
from Levenshtein import distance as levenshtein_distance
from Bio.SubsMat import MatrixInfo
import cudf

In [ ]:
for i in range(len(df_test)):
    rowProteinSequence = df_test.iloc[i].protein_sequence ## Taking a row
    LevenstreinDist = Levenshtein.editops(rowProteinSequence, base) ## Using Levenshtein
    listOutput.append(LevenstreinDist)

# Preprocessing

In [ ]:
chars = list(sorted(set(train_total)))
length = len(chars)
one_hot_map = {}

for i, c in enumerate(chars):
    one_hot_map[c] = np.eye(length)[:, i]

# convert string to one-hot
def convert_string(string):
    s = []
    for c in string:
        s.append(one_hot_map[c])
    return torch.Tensor(np.array(s))

def convert_dataset(dataset):
    data = []
    for x in dataset:
        data.append(convert_string(x))
    return data

In [ ]:
train_sequences = df_train.iloc[:,0].to_numpy()
test_sequences = df_test.iloc[:,0].to_numpy()

X = convert_dataset(train_sequences)
ph = df_train.iloc[:, 1].to_numpy().tolist()
y = df_train.iloc[:, 3].to_numpy().tolist()

X_test = convert_dataset(test_sequences)
ph_test = df_test.iloc[:, 1].to_numpy().tolist()
X_test_zipped = list(zip(X_test, ph_test))

# train-val split
X_zipped = list(zip(X, ph))
X_train_zipped, X_val_zipped, y_train, y_val = train_test_split(X_zipped, y, test_size=0.3, random_state=0)

## Set up Dataloader

In [ ]:
class EnzymeDataset(Dataset):
    def __init__(self, X_zipped, y):       
        super(EnzymeDataset, self).__init__()
        self.X_zipped = X_zipped
        self.y = y

    def __len__(self):
        '''return len of dataset'''
        return len(self.X_zipped)

    def __getitem__(self, idx):
        '''return X, y, ph values at index idx'''
        x, ph = self.X_zipped[idx]
        
        if y is None:
            return x, None, ph
        else:
            return x, self.y[idx], ph

def pad_collate(batch):
    xs = [b[0] for b in batch]
    ys = [b[1] for b in batch]
    phs = [b[2] for b in batch]
    xs_pad = pad_sequence(xs, batch_first=True, padding_value=0)
    ys = torch.Tensor(np.array(ys)).reshape(-1, 1)
    phs = torch.Tensor(np.array(phs)).reshape(-1, 1)
    return xs_pad, ys, phs

In [ ]:
train_dataset = EnzymeDataset(X_train_zipped, y_train)
test_dataset = EnzymeDataset(X_test_zipped, None)
val_dataset = EnzymeDataset(X_val_zipped, y_val)

In [ ]:
# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True,
                                           collate_fn=pad_collate)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False,
                                          collate_fn=pad_collate)

val_loader = torch.utils.data.DataLoader(dataset=val_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False,
                                          collate_fn=pad_collate)

In [ ]:
print('Train data: ', len(train_dataset))
print('Test data: ', len(test_dataset))
print('Val data: ', len(val_dataset))

In [ ]:
X, y, ph = next(iter(train_loader))

# LSTM Approach

In [ ]:
class RNN(nn.Module):
    def __init__(self, hidden_size, num_layers):
        super(RNN, self).__init__()
        self.lstm = nn.LSTM(input_size=20, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size+1, 1)
        
    def forward(self, X, ph):
        # X.shape = [batch_size, seq_len, input_size]
        out, _ = self.lstm(X)
        out = out[:, -1, :] # take last output
        out = torch.cat((out, ph), 1)
        out = self.fc(out)
        return out

In [ ]:
# warning: slow!
def spearman_score(data_loader, model):
    y_pred_list = []
    y_target_list = []
    
    model.eval()
    with torch.no_grad():
        for i, (X, y, ph) in enumerate(data_loader):
            X = X.to(device)
            y = y.to(device)
            ph = ph.to(device)
            output = model(X, ph)
            
            y_pred_list.append(output.reshape(-1).detach().cpu().numpy())
            y_target_list.append(y.reshape(-1).detach().cpu().numpy())
            
    y_pred_list = [a.tolist() for a in y_pred_list]
    y_target_list = [a.tolist() for a in y_target_list]
    y_pred = list(itertools.chain.from_iterable(y_pred_list))
    y_target = list(itertools.chain.from_iterable(y_target_list))
    model.train()
    
    return stats.spearmanr(y_pred, y_target)[0]

In [ ]:
model = RNN(64, 3).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# Train the model
model.train()
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (X, y, ph) in enumerate(train_loader):
        X = X.to(device)
        y = y.to(device)
        ph = ph.to(device)
        
        # Forward pass
        output = model(X, ph)
        loss = criterion(output, y)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

In [ ]:
score = spearman_score(val_loader, model)
print(f"Spearman coefficient on validation dataset: {score}")

# Transformer-based Encoder Approach

In [ ]:
class Model(nn.Module):
    def __init__(self, dim_feedforward=512):
        super(Model, self).__init__()
        encoder_layer = nn.TransformerEncoderLayer(d_model=221, nhead=16, dim_feedforward=dim_feedforward, batch_first=True)
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=4)
        self.fc = nn.Linear(dim_feedforward+1, 1)
        
    def forward(X, ph):
        print(X.shape)
        out = self.encoder(X)
        print(out.shape)
        #out = torch.

In [ ]:
 #model = Model()
#X = torch.Tensor()